In [1]:
!nvidia-smi

Sun Jan  1 19:04:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:1A:00.0 Off |                  Off |
| 33%   59C    P2   126W / 260W |   5701MiB / 24220MiB |     92%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:1B:00.0 Off |                  Off |
| 49%   

In [2]:
import torch
from torch.utils.data import Subset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

import numpy as np

import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from tqdm import tqdm

device: cuda


In [3]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [5]:
def get_subset_indices(dataset, percentage, seed=0):
  rng = np.random.RandomState(seed)
  targets = np.array(dataset.targets)
  num_classes = len(np.unique(targets))
  num_samples_per_class = int(percentage*len(dataset)/num_classes)

  print("num_samples = ", num_samples_per_class)

  indices = []

  for c in range(num_classes):
    class_indices = (targets == c).nonzero()[0]
    indices.extend(
        list(rng.choice(class_indices, size=num_samples_per_class, replace=False))
    )
  return indices

In [ ]:
def dataloader_with_seed_perc_conc(conc_trainset, seed, perc):
    


  train_subset = Subset(
      trainset, 
      get_subset_indices(trainset, perc, int(seed))
  )
  
  trainloader = torch.utils.data.DataLoader(
    train_subset, batch_size=256, shuffle=True, num_workers=2)

  return trainloader

In [6]:
def dataloader_with_seed_perc(trainset, seed, perc):
  train_subset = Subset(
      trainset, 
      get_subset_indices(trainset, perc, int(seed))
  )
  
  trainloader = torch.utils.data.DataLoader(
    train_subset, batch_size=256, shuffle=True, num_workers=2)

  return trainloader

In [7]:
GID_data_path = "/home/hz271/PyTorch-StudioGAN/biggan_images/samples/CIFAR10-BigGAN-DiffAug-train-2022_02_11_07_23_15/fake"

# Datasets
generated_train_dataset = torchvision.datasets.ImageFolder(
    root=GID_data_path,
    transform=transform_train
)

original_train_dataset = torchvision.datasets.CIFAR10(
    root='/home/hz271/OOD/Saved/data/', train=True, download=True, transform=transform_train)

# Concatenate
train_set = torch.utils.data.ConcatDataset(
    [generated_train_dataset, original_train_dataset]
)

test_set = torchvision.datasets.CIFAR10(
    root='/home/hz271/OOD/Saved/data/', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
for dataset in train_set.datasets:
    subset = 

Dataset ImageFolder
    Number of datapoints: 1000000
    Root location: /home/hz271/PyTorch-StudioGAN/biggan_images/samples/CIFAR10-BigGAN-DiffAug-train-2022_02_11_07_23_15/fake
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /home/hz271/OOD/Saved/data/
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )


In [12]:
trainloader = dataloader_with_seed_perc(train_set, perc = 0.001, seed = 0)

AttributeError: 'ConcatDataset' object has no attribute 'targets'

In [11]:
for batch_idx, (inputs, targets) in enumerate(trainloader):
    print("batch idx = ", batch_idx, " inputs = ", inputs, " targets = ", targets)
    break

batch idx =  0  inputs =  tensor([[[[-2.4291e+00, -2.4291e+00, -2.4291e+00,  ...,  1.1571e+00,
            1.1184e+00,  1.1184e+00],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ...,  1.1765e+00,
            1.1184e+00,  1.0990e+00],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ...,  1.1959e+00,
            1.1378e+00,  1.1184e+00],
          ...,
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -2.9672e-01,
           -3.3549e-01, -4.3242e-01],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ..., -4.4721e-02,
           -1.0288e-01, -1.4165e-01],
          [-2.4291e+00, -2.4291e+00, -2.4291e+00,  ...,  1.3434e-02,
           -1.4165e-01, -1.8042e-01]],

         [[-2.4183e+00, -2.4183e+00, -2.4183e+00,  ...,  1.4758e+00,
            1.4364e+00,  1.4168e+00],
          [-2.4183e+00, -2.4183e+00, -2.4183e+00,  ...,  1.4758e+00,
            1.4364e+00,  1.3971e+00],
          [-2.4183e+00, -2.4183e+00, -2.4183e+00,  ...,  1.4758e+00,
            1.4364e+00,  1.3971e